In [1]:
%%file myConv.py

import torch
import numpy as np
def myConv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros'):
  def Svertca(matrix):

    #groups exeption
    if (in_channels%groups != 0) or (out_channels%groups!=0):
      raise Exception('in_channels and out_channels must be divisible by groups')

    #генерация bias
    if bias == True:
      bias_val = torch.rand(out_channels)
    else:
      bias_val = torch.zeros(out_channels)

    #padding_mode
    if (padding_mode == 'zeros'):
      pad = torch.nn.ZeroPad2d(padding)
      matrix = pad(matrix)
    if (padding_mode == 'reflect'):
      pad = torch.nn.ReflectionPad2d(padding)
      matrix = pad(matrix)
    if (padding_mode == 'replicate'):
      pad = torch.nn.ReplicationPad2d(2)
      matrix = pad(matrix)
    if (padding_mode == 'circular'):
      pad = torch.nn.CircularPad2d(padding)
      matrix = pad(matrix)

    #генерация ядра
    if type(kernel_size) == tuple:
      filter = torch.rand(out_channels, in_channels//groups, kernel_size[0], kernel_size[1])
    if type(kernel_size) == int:
      filter = torch.rand(out_channels, in_channels//groups, kernel_size, kernel_size)

    #filter = b
    # bias_val = bi

    llist = []
    for l in range(out_channels):
      feature_map = np.array([]) #генерация пустой feature-map
      for i in range (0, matrix.shape[1]-((filter.shape[2]-1)*dilation+1)+1, stride): #(filter.size - 1)*dilation + 1 при delation
        for j in range (0, matrix.shape[2]-((filter.shape[3]-1)*dilation+1)+1, stride):
          summa = 0
          for c in range (in_channels//groups): #groups
            if groups>1:
              val = matrix[l*(in_channels//groups)+c][i:i+(filter.shape[2]-1)*dilation+1:dilation, j:j+(filter.shape[3]-1)*dilation+1:dilation]
            else:
              val = matrix[c][i:i+(filter.shape[2]-1)*dilation+1:dilation, j:j+(filter.shape[3]-1)*dilation+1:dilation]
            mini_sum = (val*filter[l][c]).sum()
            summa = summa + mini_sum
          feature_map = np.append(feature_map, float(summa + bias_val[l])) #bias

      llist.append(feature_map.reshape((matrix.shape[1]-((filter.shape[2]-1)*dilation+1))//stride+1, (matrix.shape[2]-((filter.shape[3]-1)*dilation+1))//stride+1))

    return np.array(llist), np.array(filter), np.array(bias_val)

  return Svertca

Overwriting myConv.py


In [2]:
%%file conftest.py

from pytest import fixture
import requests

@fixture
def session():
  s = requests.Session()
  s.headers.update({"User-Agent": "pytest requests"})
  s.verify = True
  yield s
  s.close()

Overwriting conftest.py


In [3]:
%%file test_http.py

import numpy as np
import torch
import sys
import os

py_file_location = "/content/myConv.py"
sys.path.append(os.path.abspath(py_file_location))
from myConv import myConv2d

tensor1 = torch.rand(4, 5, 6)
tensor2 = torch.rand(3, 28, 28)
tensor3 = torch.rand(1, 1, 1)


def test_1(session):
  myFunction = myConv2d(in_channels=4, out_channels=2, kernel_size=3, stride=1, padding=0, dilation=1, groups=2, bias=True, padding_mode='zeros')
  result, kernel, bias_val = myFunction(tensor1)
  torchFunction = torch.nn.Conv2d(in_channels=4, out_channels=2, kernel_size=3, stride=1, padding=0, dilation=1, groups=2, bias=True, padding_mode='zeros')
  torchFunction.weight.data = torch.tensor(kernel)
  torchFunction.bias.data = torch.tensor(bias_val)
  myResult = str(np.round(result, 2))
  torchResult = str(np.round(np.array(torchFunction(tensor1).data), 2))
  assert torchResult == myResult


def test_2(session):
  myFunction2 = myConv2d(in_channels=3, out_channels=1, kernel_size=3, stride=1, padding=0, dilation=2, groups=1, bias=True, padding_mode='zeros')
  result2, kernel2, bias_val2 = myFunction2(tensor2)
  torchFunction2 = torch.nn.Conv2d(in_channels=3, out_channels=1, kernel_size=3, stride=1, padding=0, dilation=2, groups=1, bias=True, padding_mode='zeros')
  torchFunction2.weight.data = torch.tensor(kernel2)
  torchFunction2.bias.data = torch.tensor(bias_val2)
  myResult = str(np.round(result2, 2))
  torchResult = str(np.round(np.array(torchFunction2(tensor2).data), 2))
  assert torchResult == myResult


def test_3(session):
  myFunction3 = myConv2d(in_channels=1, out_channels=1, kernel_size=1, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
  result3, kernel3, bias_val3 = myFunction3(tensor3)
  torchFunction3 = torch.nn.Conv2d(in_channels=1, out_channels=1, kernel_size=1, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
  torchFunction3.weight.data = torch.tensor(kernel3)
  torchFunction3.bias.data = torch.tensor(bias_val3)
  myResult = str(np.round(result3, 2))
  torchResult = str(np.round(np.array(torchFunction3(tensor3).data), 2))
  assert torchResult == myResult



Overwriting test_http.py


In [4]:
!pytest

============================= test session starts =============================
platform win32 -- Python 3.9.13, pytest-7.1.2, pluggy-1.0.0
rootdir: C:\Users\arina\smz
plugins: anyio-3.5.0
collected 3 items

test_http.py ...                                                         [100%]

============================== 3 passed in 6.83s ==============================


In [8]:
import numpy as np
import torch
import sys
import os

py_file_location = "/content/myConv.py"
sys.path.append(os.path.abspath(py_file_location))
from myConv import myConv2d

tensor4 = torch.rand(4, 5, 5)

myFunction4 = myConv2d(in_channels=4, out_channels=2, kernel_size=3, stride=2, padding=2, dilation=1, groups=2, bias=True, padding_mode='zeros')
result4, kernel4, bias_val4 = myFunction4(tensor4)
torchFunction4 = torch.nn.Conv2d(in_channels=4, out_channels=2, kernel_size=3, stride=2, padding=2, dilation=1, groups=2, bias=True, padding_mode='zeros')
torchFunction4.weight.data = torch.tensor(kernel4)
torchFunction4.bias.data = torch.tensor(bias_val4)

print(result4)
print('===================================')
print(np.array(torchFunction4(tensor4).data))

[[[1.24100769 1.76235616 1.84644806 1.22685492]
  [2.98437047 6.4995079  5.23831654 2.16040158]
  [2.70378065 5.06630135 5.60799313 2.08769846]
  [1.12965584 1.87915885 2.81096125 1.29868186]]

 [[0.85043979 1.33088338 1.32333064 1.17367244]
  [1.85647118 4.62818146 4.97800732 1.94978023]
  [2.34088397 5.68943977 5.57706213 1.59353518]
  [1.15338933 2.81359816 2.25473499 0.81117088]]]
[[[1.2410077 1.7623563 1.8464481 1.2268549]
  [2.9843705 6.499508  5.2383165 2.1604018]
  [2.7037807 5.0663013 5.607994  2.0876985]
  [1.1296558 1.879159  2.8109612 1.2986819]]

 [[0.8504398 1.3308834 1.3233306 1.1736724]
  [1.8564711 4.628181  4.9780073 1.9497803]
  [2.340884  5.6894393 5.5770617 1.5935352]
  [1.1533892 2.8135982 2.254735  0.8111709]]]


In [5]:
import numpy as np
import torch
import sys
import os

py_file_location = "/content/myConv.py"
sys.path.append(os.path.abspath(py_file_location))
from myConv import myConv2d

tensor1 = torch.rand(8, 5, 6)

myFunction = myConv2d(in_channels=8, out_channels=4, kernel_size=3, stride=1, padding=0, dilation=1, groups=4, bias=True, padding_mode='zeros')
result, kernel, bias_val = myFunction(tensor1)
torchFunction = torch.nn.Conv2d(in_channels=8, out_channels=4, kernel_size=3, stride=1, padding=0, dilation=1, groups=4, bias=True, padding_mode='zeros')
torchFunction.weight.data = torch.tensor(kernel)
torchFunction.bias.data = torch.tensor(bias_val)

print(result)
print('===================================')
print(np.array(torchFunction(tensor1).data))

[[[7.95807743 5.40207386 6.84529543 7.35895252]
  [8.12699604 6.53488064 6.11109924 7.02044964]
  [7.71687603 6.13926697 5.36437511 6.74329329]]

 [[5.21243382 3.98044205 4.56722546 3.94700956]
  [4.43468094 4.32671642 3.83685684 3.98184586]
  [3.97997761 3.50106382 4.49686432 3.51984549]]

 [[5.71537352 5.43403864 6.05418253 5.78864956]
  [5.31311941 5.65596151 5.868855   4.49289322]
  [5.5946641  5.91159487 4.91239071 4.61102772]]

 [[4.18876743 5.21671343 4.80044031 5.51270962]
  [5.17841578 5.43035889 5.15033531 4.77567434]
  [6.12616253 5.76851654 5.50071192 5.486022  ]]]
[[[7.958077  5.4020734 6.845296  7.3589525]
  [8.126996  6.5348797 6.111099  7.02045  ]
  [7.7168765 6.1392674 5.3643756 6.743293 ]]

 [[5.2124343 3.980442  4.5672255 3.947009 ]
  [4.4346814 4.326716  3.8368568 3.9818463]
  [3.9799776 3.5010638 4.4968643 3.5198455]]

 [[5.7153735 5.4340386 6.0541816 5.7886496]
  [5.313119  5.655961  5.868855  4.492893 ]
  [5.594664  5.9115944 4.9123907 4.6110277]]

 [[4.1887674 5

In [6]:
import numpy as np
import torch
import sys
import os
py_file_location = "/content/myConv.py"
sys.path.append(os.path.abspath(py_file_location))
from myConv import myConv2d

tensor2 = torch.rand(3, 10, 10)

myFunction2 = myConv2d(in_channels=3, out_channels=1, kernel_size=3, stride=1, padding=0, dilation=2, groups=1, bias=True, padding_mode='zeros')
result2, kernel2, bias_val2 = myFunction2(tensor2)
torchFunction2 = torch.nn.Conv2d(in_channels=3, out_channels=1, kernel_size=3, stride=1, padding=0, dilation=2, groups=1, bias=True, padding_mode='zeros')
torchFunction2.weight.data = torch.tensor(kernel2)
torchFunction2.bias.data = torch.tensor(bias_val2)

print(result2)
print('===================================')
print(np.array(torchFunction2(tensor2).data))

[[[ 9.4219656  10.47064114  8.05049801  8.20273113  9.50746822
    7.2307477 ]
  [ 9.43639374  7.60696936  8.02908421  6.88420534  9.48032951
    7.04393435]
  [ 9.52552128  9.40228653  7.43158674  8.48575401  9.8943758
    7.49710989]
  [ 9.16875935  6.58858442  8.3532362   6.96097803  9.41242695
    7.16045523]
  [ 8.64023495  8.5577755   8.44954586  8.76201057  9.32271767
    8.1281786 ]
  [ 7.80711508  6.7029109   8.34368229  7.77376366  9.18826199
    6.84338713]]]
[[[ 9.421965  10.470641   8.050498   8.202731   9.507469   7.230748 ]
  [ 9.436396   7.606969   8.029084   6.8842053  9.4803295  7.0439343]
  [ 9.525521   9.402286   7.4315863  8.485754   9.894376   7.49711  ]
  [ 9.16876    6.5885844  8.353235   6.960978   9.412427   7.1604557]
  [ 8.640235   8.557776   8.449547   8.762012   9.322717   8.128179 ]
  [ 7.8071156  6.702911   8.343682   7.773762   9.188262   6.843386 ]]]


In [7]:
import numpy as np
import torch
import sys
import os

py_file_location = "/content/myConv.py"
sys.path.append(os.path.abspath(py_file_location))
from myConv import myConv2d

tensor3 = torch.rand(1, 1, 1)

myFunction3 = myConv2d(in_channels=1, out_channels=1, kernel_size=1, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
result3, kernel3, bias_val3 = myFunction3(tensor3)
torchFunction3 = torch.nn.Conv2d(in_channels=1, out_channels=1, kernel_size=1, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
torchFunction3.weight.data = torch.tensor(kernel3)
torchFunction3.bias.data = torch.tensor(bias_val3)

print(result3)
print('===================================')
print(np.array(torchFunction3(tensor3).data))

[[[0.17242639]]]
[[[0.17242639]]]
